In [1]:
#code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested
